In [2]:
import pandas as pd
import xarray as xr
from fooof import FOOOFGroup
import matplotlib.pyplot as plt
import numpy as np

from lambic.utils import filter_xarray
from lambic.spectrum import psd_fooof, fooof2pandas


In [8]:

# the analysis is focusing on parietal parts of the brain

def add_columns(df, xdf):
    temp_df = pd.DataFrame()
    temp_df["ROI"] = xdf.coords['roi_names'].values
    temp_df.insert(0, 'ID', temp_df.index)
    temp_df["subjectID"] = xdf.coords['sub'].values
    return df.join(temp_df.set_index('ID'), on='ID')

# TODO
def plot_spectra(fg, df):
    for y in range(np.shape(fg.power_spectra)[0]):
        fm = fg.get_fooof(y, regenerate=True)
        # Check results and visualize the extracted model
        fm.report()
        display(df[df['ID'] == y] )
        plt.show()

def iterate_subjects_fooof(roi_names_set, plot = None):
    ds = roi_names_set.to_dict(data=False)
    dict_dims = dict(zip(ds["coords"].keys(),roi_names_set.indexes.values()))
    pandas_list = []

    for subject  in dict_dims["sub"][:]:
        temp_xarray = roi_names_set.sel(sub=subject)
        spectra = temp_xarray.__xarray_dataarray_variable__.data
        frequencies = np.array(dict_dims["freqs"])
        fparams =  FOOOFGroup(
            peak_width_limits=[2, 12],
            min_peak_height=0.1,
            max_n_peaks=6,
            aperiodic_mode="fixed",
        )
        fg = psd_fooof(frequencies, spectra,  fparams,freq_range= [2, 40])
        fg.save(f'fg_{subject}_parietal', file_path='data/results', save_results=True)  # type: ignore
        df = fooof2pandas(fg)
        df_combined = add_columns( df, temp_xarray)
        df_combined.drop('ID',  inplace=True, axis=1)
        pandas_list.append(df_combined)
        if plot == True:
            plot_spectra(fg,  df_combined)
    return(pd.concat(pandas_list))

In [9]:

contains = ["inferiorparietal", "postcentral", "superiorparietal", "supramarginal"]
coordinate = 'roi_names'

ds52 = xr.open_dataset("data/freq52.nc")
roi_names_set52 = filter_xarray(ds52, coordinate, contains)
df52 = iterate_subjects_fooof(roi_names_set52);

ds128 = xr.open_dataset("data/freq128.nc")
roi_names_set128 = filter_xarray(ds128, coordinate, contains)
df128 = iterate_subjects_fooof(roi_names_set128);

full_df = pd.concat([df52,df128])
full_df.to_csv('data/ROI_offset_exponent.csv', index = False, header=True)



['inferiorparietal L', 'inferiorparietal R', 'postcentral L', 'postcentral R', 'superiorparietal L', 'superiorparietal R', 'supramarginal L', 'supramarginal R']


Running FOOOFGroup: 100%|██████████| 8/8 [00:01<00:00,  4.70it/s]

['inferiorparietal L', 'inferiorparietal R', 'postcentral L', 'postcentral R', 'superiorparietal L', 'superiorparietal R', 'supramarginal L', 'supramarginal R']



Running FOOOFGroup: 100%|██████████| 8/8 [00:01<00:00,  4.57it/s]
